In [10]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup

In [33]:
# Driver execute
driver = webdriver.Chrome('/Users/dangchan/Documents/chromedriver')
driver.get("https://map.naver.com/v5/search")
time.sleep(2)

# popup close
driver.find_element(by=By.XPATH, value= "/html/body/app/layout/dynamic-content-outlet/promotion-popup/div/button").click()

/var/folders/6r/ndnspf4n6n7_m8rb9bs7qdvr0000gp/T/ipykernel_14602/3288361001.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/dangchan/Documents/chromedriver')


In [41]:
# search box
search_box = driver.find_element(by=By.CSS_SELECTOR, value="div.input_box>input.input_search")
search_box.send_keys("대전광역시 ATM")
time.sleep(2)
search_box.send_keys(Keys.ENTER)
time.sleep(2)

In [ ]:
for p in range(20):
    # 5초 delay
    time.sleep(2)
    
    js_script = "document.querySelector(\"body > app > layout > div > div.container > div.router-output > "\
                "shrinkable-layout > search-layout > search-list > search-list-contents > perfect-scrollbar\").innerHTML"
    raw = driver.execute_script("return " + js_script)

    html = BeautifulSoup(raw, "html.parser")

    contents = html.select("div > div.ps-content > div > div > div .item_search")
    for s in contents:
        search_box_html = s.select_one(".search_box")

        name = search_box_html.select_one(".title_box .search_title .search_title_text").text
        print("식당명: " + name)
        try:
            phone = search_box_html.select_one(".search_text_box .phone").text
        except:
            phone = "NULL"
        print("전화번호: " + phone)
        address = search_box_html.select_one(".ng-star-inserted .address").text
        print("주소: " + address)

        print("--"*30)
    # 다음 페이지로 이동
    try:
        next_btn = driver.find_element_by_css_selector("button.btn_next")
        next_btn.click()
    except:
        print("데이터 수집 완료")
        break

# 크롭 웹페이지를 닫음
driver.close()